# Constructions of mutually unbiased bases.
## Andreas Klappenecker and Martin Röttler

Klappenecker presents two different constructions of MUBs for the case of odd prime power dimensions. And for the case of even prime power dimension, he uses exponentials sums over Galois `rings`.

Let $\mathbb F_q$ be a finite field with odd characteristic $p$. Define for each $x \in \mathbb F_q$ the additive character $\mathbb F_q \to \mathbb C^{\times}$ by

$$
y \mapsto \omega_p^{\text{tr}(xy)},
$$
where $\omega_p = \exp(2\pi i / p)$. Assume some order of the elements of the finite field.

**Theorem** Let $\mathbb F_q$ be the finite field with odd characteristic $p$. Denote $B_a = \{v_{a,b} | b \in \mathbb F_q\}$ the set of vectors given by

$$
v_{a,b} = q^{-1/2} \left(\omega_p^{\text{tr}(ax^2+bx)}\right)_{x \in \mathbb F_q}.
$$

The standard basis and the sets $B_a$, with $a \in \mathbb F_q$, form a set of $q+1$ mutually unbiased bases of $\mathbb C^q$.

In [94]:
from sage.all import *
reset()

In [96]:
p = 3
n = 2
q = p**n

Fp = PolynomialRing(GF(p), 't'); t = Fp.gen()
Fq = GF(q, 'w', modulus=t**2+t+2); w = Fq.gen()
Fq

Finite Field in w of size 3^2

In [101]:
B = [identity_matrix(SR, q)]
for a in Fq:
    Ba = zero_matrix(SR, q, q)
    for j, b in enumerate(Fq):
        for k, x in enumerate(Fq):
            Ba[k, j] = exp(2 * pi * i * lift((a * x**2 + b * x).trace()) / p)
    B.append(Ba / sqrt(q))

In [102]:
# sanity check
(vector(B[3][:,2].transpose()) * vector(B[1][:,2])).abs().simplify()

1/3

For even prime powers, Klappenecker and Martin mention that there proof (using Gauss sums) does not work in even characteristic. But it turns out that exponential sums over Galois rings serve as a substitute.

Let $\mathbb Z_4$ denote the residue class ring of integers modulo $4$. Denote $\langle 2 \rangle$ the ideal generated by $2$ in $\mathbb Z_4[x]$. Let $h(x)$ be a monic basic primitive polynomial of degree $n$. The ring $GR(4,n) = \mathbb Z_4[x] / \langle h(x) \rangle$ is called the Galois ring of degree $n$ over $\mathbb Z_4$. Such a ring has $4^n$ elements. Any element $r \in GR(4,n)$ can be uniquely written in the form $r = a + 2b$, where $a, b \in \mathcal T_n = \{0,1,\xi,\ldots,\xi^{2^n-2}\}$.

The automorphism $\sigma: GR(4,n) \to GR(4,n)$ defined by $\sigma(a+2b) = a^2 + 2b^2$ is called the Frobenius automorphism. This map leaves the elements of the ring $\mathbb Z_4$ fixed. All automorphisms of $GR(4,n)$ are of the form $\sigma^k$ for some integer $k \geq 0$. Define the trace map $\text{tr} : GR(4,n) \to \mathbb Z_4$ as

$$
\text{tr}(x) = \sum_{k=0}^{n-1} \sigma^k(x).
$$

**Theorem.** Let $GR(4,n)$ be a finite Galois ring with Teichmüller set $\mathcal T_n$. For $a \in \mathcal T_n$, denote by $M_a = \{v_{a,b} | b \in \mathcal T_n\}$ the set of vectors given by

$$
v_{a,b}
= 2^{-n / 2} \left(
    \exp\left(2\pi i / 4 \cdot \text{tr}((a+2b)x)\right)
\right)_{x \in \mathcal T_n}.
$$

The standard basis and the sets $M_a$, with $a \in \mathcal T_n$, form a set of $2^n + 1$ mutually unbiased bases in $\mathbb C^{2^n}$.

In [ ]:
reset()

In [137]:
p = 2
n = 2
q = p**n

# GR(4,2)
R  = PolynomialRing(Integers(4), 't'); t = R.gen()
GR = R.quotient(t**2+t+1, 'w'); w = GR.gen()
GR

Univariate Quotient Polynomial Ring in w over Ring of integers modulo 4 with modulus t^2 + t + 1

In [118]:
GR.cardinality()

16

In [133]:
# def generalizedFrobenius(a, b):
#     return a**2 + GR(2)*b**2

def ringTrace(a, b):
    s = GR(0)
    for k in range(n):
        s += a**(2**k) + GR(2)*b**(2**k)
    return s 

for a in GR:
    for b in GR:
        if not ringTrace(a, b) == a + GR(2)*b + a**2 + GR(2)*b**2:
            raise Error('Trace not well defined!')
print('Trace defined!')

Trace defined!


In [139]:
T = [0] + [w**k for k in range(2**n - 2 + 1)]
T

[0, 1, w, 3*w + 3]

In [202]:
mubs = [identity_matrix(q)]
for a in T:
    Ba = zero_matrix(SR, q, q)
    for j, b in enumerate(T):
        for k, x in enumerate(T):
            Ba[k, j] = exp( 2*pi*I/4 * Integer(lift( ringTrace(a*x, b*x) )) ) * 2**(-n/2)
    mubs.append(Ba)

In [203]:
def testMubs(mubs):
    for B in mubs:
        for V in mubs:
            if B != V:
                for j in range(4):
                    for k in range(4):
                        r = (transpose(B[:,j]) * V[:,k]).norm().abs()**2
                        if r != 1/4:
                            print(B)
                            print(V)
                            print(r)
                            raise Exception('Not MUBs!')
    return True

In [204]:
testMubs(mubs) # are they MUBs?

True